In [1]:
%load_ext tikzmagic
%matplotlib inline

# 회귀분석을 위한 준비

제대로 된 회귀분석을 하기 위해서는 분석 이전 단계에서의 다음과 같은 준비작업이 필요하다. 

1. 변수의 결정
2. 데이터의 수집
3. 데이터의 변환
4. 데이터 전처리

이 절에서는 이러한 준비 과정에 대해 설명한다. 

## 변수의 결정

회귀분석을 하기 위한 가장 첫번째 단계는 종속변수와 독립변수를 결정하는 것이다. 즉, 어떤 데이터(종속변수)에 영향을 주는 요소가 무엇무엇(독립변수)이 있을지 찾아내야 한다. 보통 분석의 대상이 되는 종속변수는 결정되어 있는 경우가 많지만 영향을 비치는 독립변수는 분석을 수행하는 사람이 찾아야 한다. 당연하게도 목표가 되는 종속변수에 많은 영향을 끼지는 중요한 독립변수를 찾지 못하면 좋은 분석결과는 나올 수 없다. 이 부분은 컴퓨터나 수학이 대신해 줄 수 없는 부분으로 데이터가 속해있는 분야의 지식과 노하우 그리고 직관에 의해 결정된다. 

변수가 결정되면 결정된 종속변수를 $y$, 결정된 $K$개의 독립변수를 $x_1, x_2, \ldots, x_K$라고 이름붙인다. 

## 데이터의 수집

변수가 결정되면 해당 변수들의 표본값을 수집하는 과정이 필요하다. 사실 실제 데이터 분석 실무에서는 이 과정에 들어가는 시간과 노력이 대부분을 차지할 정도로 중요한 과정이나 여기에서는 이미 수집된 데이터가 존재한다고 가정한다.

데이터는 보통 다음과 같은 형식으로 수집된다. 여기에서는 수집된 데이터의 갯수가 $N$개라고 가정한다. $i$번째 레코드의 종속변수 값을 $y_i$, $i$번째 레코드의 $j$번째 독립변수 값을 $x_{ij}$라고 표시한다. 


|  | 종속변수 $y$ 값 | 독립변수 $x_1$ 값 | 독립변수 $x_2$ 값 | ... | 
|-|-|-|-|-|
| 표본 1 | 수치 $y_1$ | 수치 $x_{11}$ | 수치 $x_{12}$ | ... |
| 표본 2 | 수치 $y_2$ | 수치 $x_{21}$ | 수치 $x_{22}$ | ... |
| 표본 3 | 수치 $y_3$ | 수치 $x_{31}$ | 수치 $x_{32}$ | ... |
| : | : | : | : | ... |
| 표본 N | 수치 $y_N$ | 수치 $x_{N1}$ | 수치 $x_{N2}$ | ... |



## 데이터를 벡터와 행렬 형식으로 변환

데이터가 준비되면 다음과 같이 

* 종속변수는 벡터 $\mathbf{y}$ 형식으로 
* 독립변수는 행렬 $\mathbf{X_0}$ 형식으로 

변환한다. 

$$ 
\mathbf{y}
=
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_N \\
\end{bmatrix}
$$

$$ 
\mathbf{X_0}
=
\begin{bmatrix}
x_{11} & x_{12} & \cdots & x_{1K} \\
x_{21} & x_{22} & \cdots & x_{2K} \\
\vdots & \vdots & \ddots & \vdots \\
x_{N1} & x_{N2} & \cdots & x_{NK} \\
\end{bmatrix}
$$

## 데이터의 전처리

이렇게 수집 및 정리된 데이터 값에 대해 다음과 같은 전처리를 거치게 된다.

* 잘못된 데이터가 있는지 검사하고 해당 레코드 삭제하거나 정상적인 데이터로 대체
* 누락된 데이터가 있는지 검사하고 해당 레코드를 삭제하거나 가상의 데이터로 대체
* 상수항 추가
* 독립변수 중에 중복 혹은 종속적인 변수가 존재하는지 확인
* 독립변수 단위 조정


### 잘못된 데이터 처리

### 누락된 데이터 처리

### 상수항 추가

### 독립변수의 독립성 확인

 행렬 $\mathbf{X_0}$의 랭크 검사

### 독립변수 단위 조정

스케일링